In [ ]:
Brand Voice using Tuned foundation model

Your brand's voice is its soul - the way it speaks to the world. This nptebook will become your essential toolkit for crafting and refining
a distinct voice of the brand for all your content creation efforts. It's designed to be a living document, guiding you in translating 
abstract brand values into tangible communication

On Vertex AI, tuning allows you to customize a foundation model for more specific tasks or knowledge domains.

While the prompt design is excellent for quick experimentation, if training data (examples) is available, tuning a model enables you to
customize the model for the characteristics of brand you want to project

Objective

This tutorial teaches you how to tune a foundation model on new unseen data and you will use the following google cloud products:
    1. Vertex AI Generative AI Studio
    2. Vertex AI pipelines
    3. Vertex AI model registry
    4. Vertex AI Endpoints
    
This steps performed include 
    1. Upload training data 
    2. Create a pipeline job
    3. Inspect your model on Vertex AI Model Registry
    4. Get predictions from your tuned model
    
    
Quota

important: Tuning the text-bison@002 model uses the tpu-v3-8 training resources and the accompanying quotas rom your google
Cloud project. Each project has a default quota of eight v3-8 cores, which allows for one to two concurrent tuning jobs. If you want to run more concurrent jobs you need
to run more concurrent jobs you need to request additional quota via Quotas page

Costs

This tutorial uses billable components of Google Cloud:
    
    1. Vertex AI Generative AI Studio
    
Learn about Vertex AI pricing and use the Pricing Calculator to generate a cost estimate based on the projected usage

Install Vertex AI SDK

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user --quiet

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Authenticating the notebook environment

1. while using colab, uncomment the cell below & then continue
2. While using Vertex AI Workbench, check instruction in https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env 

In [ ]:
from google.colab import auth
auth.authenticate_user()

Set the Project ID

Update project ID using gcloud or use https://support.google.com/googleapi/answer/7014113

In [ ]:
PROJECT_ID = "GOOGLE_CLOUD_PROJECT_HERE"

! gcloud config set project {PROJECT_ID}

Create the bucket

Now we have to create the bucket that we will store the tuning data. To avoid name collissions b/w users on
resources created, generate a UUID for each instance session and append it to the name of the resources 
created in this tutorial

In [ ]:
import random
import string

#Generate a uuid of a specified length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))

UUID = generate_uuid()

Choose a bucket name and update BUCKET_NAME parameter.

In [ ]:
BUCKET_NAME = "genai-mkt-dev/tune-dataset"
BUCKET_URI = f"gs://{BUCKET_NAME}"
REGION = "us-central1"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "<BUCKET_NAME>":
    BUCKET_NAME = "vertex-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

Only if the bucket doesn't already exist: Run the following cell to create Cloud Storage bucket

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally validate access to the Cloud Storage bucket by examining its contents

In [ ]:
! gsutil ls -a1 $BUCKET_URI

Import Libraries

Colab only: Run the cell to initialize the Vertex AI SDK. In Vertex AI, it isn't required

In [ ]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
from typing import Union

import pandas as pd

from google.cloud import aiplatform
from vertexai.language_models import TextGenerationModel

Tune the Model

Now it's time to create a tuning job. Tune a foundation modelby creating a pipeline job using Generative AI Studio, cURL, or the Python SDK. Here we will be using Python SDK. We will be using a Q&A with a context dataset in JSON format

Training Data

Your model tuning dataset must be a JSONL format where each line ontains a single training example. You must make 
sure that you include instructions

Upload to cloud storage bucket and add filenames below

In [ ]:
training_data_filename = "tune_data_brand_voice.json1"

In [ ]:
evaluation_data_filename = "tune_eval_data_brand_voice.json1"

You can check to make sure that the files are available in your Google cloud storage bucket:

In [ ]:
! gsutil ls -a1 $BUCKET_URI

In [ ]:
TRAINING_DATA_URI = f"${BUCKET_URI}/{training_data_filename}"
EVALUATION_DATA_URI = f"${BUCKET_URI}/{evaluation_data_filename}"

Model Tuning

Now it's time to tune a model. You will use the Vertex AI SDK to submit our tuning job.

Recommended Tuning Configurations

Extractive QA:
1. Make sure that train dataset size is 100+
2. Training steps[100-500]. Can try more than one value to get the best performance on a particular dataset (e.g 00, 200, 500)

In [ ]:
display_name = "Adapter Tuning Voice - "

tensorboard = aiplatform.Tensorboard.create(
    display_name = display_name,
    project = PROJECT_ID,
    location = REGION
)

print(tensorboard.display_name)
print(tensorboard.resource_name)

In [ ]:
tensorboard_id = tensorboard.resource_name.split("tensorboards/")[-1]
print(tensorboard_id)

In [ ]:
MODEL_NAME = f"genai-tuned-model-{UUID}"
TRAINING_STEPS = 100

In [ ]:
pipeline_arguments = {
    "model_display_name": MODEL_NAME,
    "location": REGION,
    "large_model_reference": "text-bison@002",
    "project": PROJECT_ID,
    "train_steps": TRAINING_STEPS,
    "dataset_uri": TRAINING_DATA_URI,
    "evaluation_interval": 20,
    "evaluation_data_uri": EVALUATION_DATA_URI,
    "tensorboard_resource_id": tensorboard_id
}

pipeline_root = f"{BUCKET_URI}/{MODEL_NAME}"
template_path = "https://us-kfp.pkg.dev/ml-pipeline/large-language-model-pipelines/tune-large-model/v2.0.0"


In [ ]:
def tuned_model(
    project_id: str,
    location: str,
    template_path: str,
    model_display_name: str,
    pipeline_arguments: str,
):
    """Prompt-tune a new model, based on a prompt-response data.
    
    "training_data" can be either the GCS URI of a file formatted in JSONL format
    (for example: training_data=f'gs://{bucket}/{filename}.jsonl'), or a pandas
    DataFrame. Each training example should be JSONL record with 2 keys, for 
    example:
       {
           "input_text": <input_prompt>,
           "output_text": <associated output>
       },
       
    Args:
       project_id: GCP project ID, used to initialise aiplatform
       location: GCP region, used to initialise aiplatform
       template_path: path to the template
       model_display_name: Name for your model
       pipeline_arguments: arguments used during pipeline runtime
    
    """
    
    aiplatform.init(project=project_id, location=location)
    
    from google.cloud.aiplatform import PipelineJob
    
    job = PipelineJob(
        template_path=template_path,
        display_name=model_display_name,
        parameter_values=pipeline_arguments,
        location=REGION,
        pipeline_root=pipeline_root,
        enable_caching=True
    )
    
    return job

Next, it's time to start tuning the job

DISCLAIMER: tuning and deploying a model takes time

In [ ]:
job = tuned_model(PROJECT_ID, REGION, template_path, MODEL_NAME, pipeline_arguments)

In [ ]:
job.submit()

Following link above, you can view your pipeline run. As you can see in the screenshot below, it will execute the following steps

1. Validation
2. Export managed dataset
3. Convert JSONL to TFRecord
4. Large language model tuning
5. Upload LLM model

job.state let's you check the state of your pipeline

In [ ]:
job.state

View your tuned foundational model on vertex AI model registry

When your tuning job is finished, your model will be available on Vertex AI Model registry. The following Python SDK sample shows you how to list tuned models

In [ ]:
def list_tuned_models(project_id, location):
    aiplatform.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@002")
    tuned_model_names = model.list_tuned_model_names()
    print(tuned_model_names)

In [ ]:
list_tuned_models(PROJECT_ID, REGION)

You can also use the Google Cloud Console UI to view all of your model in Vertex AI Model Registry. Below you can see an example of a tuned foundational model available on Vertex AI Model Registry.

Use your tuned model to get predictions

Now it's time to get predictions. First you need to get the latest tuned model from the Vertex AI Model Registry.

In [ ]:
def fetch_model(project_id, location):
    aiplatform.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@002")
    list_tuned_models = model.list_tuned_model_names()
    tuned_model = list_tuned_models[0]
    
    return tuned_model

In [ ]:
deployed_model = fetch_model(PROJECT_ID, REGION)
deployed_model = TextGenerationModel.get_tuned_model(deployed_model)

Now we can start send a prompt to the API. Feel free to update the following prompt

In [ ]:
PROMPT = """
Theme: Sales of new women's handbags at Cymbal\nUsing the Brand Voice, generate a personalized email with the theme mentioned above for the user 
"""

In [ ]:
print(deployed_model.predict(PROMPT))

In [ ]:
foundation_model = TextGenerationModel.from_pretrained("text-bison@002")
print(foundation_model.predict(PROMPT))